<a href="https://colab.research.google.com/github/williamsueyoshi/unicamp_feec_ia368_120282/blob/main/exercicio_selecao_120282.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports de bibliotecas

In [1]:
import math
import nltk
import os
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Import e tratamento da CISI Collection

In [2]:
!cd '/content'

In [3]:
!tar xf "/content/cisi.tar.gz"

In [4]:
# Trata arquivo CISI.ALL (contém os documentos a serem pesquisados)

with open('/content/CISI.ALL', 'r') as f:
    input_completo = f.read()

# Lista com documentos separados e ID no formato certo para tratamento
docs = ['\n.I\n' + i for i in input_completo.split('.I ')[1:]]

# Legenda dos campos do arquivo (metadados)
metadata_dict = {
    '.I': 'ID',
    '.T': 'Titulo',
    '.A': 'Autor',
    '.B': 'Referencia',
    '.W': 'Texto',
    '.X': 'Referencia Cruzada',
    }

# String com simbolo dos campos para separação no dicionário
metadata_simbolos_str = ''
for simbolo in metadata_dict.keys():
    metadata_simbolos_str += ('\n' + simbolo + '|')
metada_simbolo_str = metadata_simbolos_str[:-1]

# Cria dicionário com ID dos documentos na chave
docs_dict = {}
for doc in docs:

    # Separa string do documento completo nos campos dos metadados
    doc_dict = {}
    for simbolo_campo, nome_campo in metadata_dict.items():
        if ('\n' + simbolo_campo) in doc:
            texto_campo = doc.split('\n' + simbolo_campo, 1)[1]
            texto_campo = re.split(metada_simbolo_str, texto_campo)[0]
            texto_campo = texto_campo.replace('\n', ' ').strip()
            doc_dict[nome_campo] = texto_campo

    docs_dict[int(doc_dict['ID'])] = doc_dict

In [5]:
# Trata arquivo CISI.QRY (contém queries a serem executadas)

with open('/content/CISI.QRY', 'r') as f:
    input_completo = f.read()

# Lista com queries separadas e ID no formato certo para tratamento
qrys = ['\n.I\n' + i for i in input_completo.split('.I ')[1:]]

# Legenda dos campos do arquivo (metadados)
metadata_dict = {
    '.I': 'ID',
    '.T': 'Titulo',
    '.A': 'Autor',
    '.B': 'Referencia',
    '.W': 'Texto',
    }

# String com simbolo dos campos para separação no dicionário
metadata_simbolos_str = ''
for simbolo in metadata_dict.keys():
    metadata_simbolos_str += ('\n' + simbolo + '|')
metada_simbolo_str = metadata_simbolos_str[:-1]

# Cria dicionário com ID das queries na chave
qrys_dict = {}
for qry in qrys:

    # Separa string do documento completo nos campos dos metadados
    qry_dict = {}
    for simbolo_campo, nome_campo in metadata_dict.items():
        if ('\n' + simbolo_campo) in qry:
            texto_campo = qry.split('\n' + simbolo_campo, 1)[1]
            texto_campo = re.split(metada_simbolo_str, texto_campo)[0]
            texto_campo = texto_campo.replace('\n', ' ').strip()
            qry_dict[nome_campo] = texto_campo

    qrys_dict[int(qry_dict['ID'])] = qry_dict

In [6]:
# Trata arquivo CISI.REL (contém prova real do resultado das queries)

with open('/content/CISI.REL', 'r') as f:
    input_completo = f.read()

# Lista com retorno esperado por ID (um doc_id por linha, repete várias qry_id)
rels = input_completo.split('\n')[:-1]

# Cria dicionário com ID das queries na chave
rels_dict = {}
for rel in rels:

    # Atribui cada linha do REL à query ID correspondente
    rel = rel.lstrip().split('\t0')[0]
    qry_id = rel.split(' ', 1)[0]
    doc_id = rel.split(' ', 1)[1].strip()

    if int(qry_id) not in rels_dict:
        rels_dict[int(qry_id)] = {
            'Query ID': qry_id,
            'Doc IDs': [],
        }

    rels_dict[int(qry_id)]['Doc IDs'].append(doc_id)

## Verificação dos dicionários criados, contendo os dados dos arquivos .ALL, .QRY e .REL

In [7]:
docs_dict[1]

{'ID': '1',
 'Titulo': '18 Editions of the Dewey Decimal Classifications',
 'Autor': 'Comaromi, J.P.',
 'Texto': "The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad.",
 'Referencia Cruzada': '1\t5\t1 92\t1\t1 262\t1\t1 556\t1\t1 1004\t1\t1 1024\t1\t1 1024\t1\t1'}

In [8]:
qrys_dict[1]

{'ID': '1',
 'Texto': 'What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?'}

In [9]:
rels_dict[1]

{'Query ID': '1',
 'Doc IDs': ['28',
  '35',
  '38',
  '42',
  '43',
  '52',
  '65',
  '76',
  '86',
  '150',
  '189',
  '192',
  '193',
  '195',
  '215',
  '269',
  '291',
  '320',
  '429',
  '465',
  '466',
  '482',
  '483',
  '510',
  '524',
  '541',
  '576',
  '582',
  '589',
  '603',
  '650',
  '680',
  '711',
  '722',
  '726',
  '783',
  '813',
  '820',
  '868',
  '869',
  '894',
  '1162',
  '1164',
  '1195',
  '1196',
  '1281']}

## Funções para execução da busca para cada query

In [10]:
# Função para pré-processar o texto do documento
def preprocess_text(text):

    # Remove a pontuação e transforma tudo em minúsculas
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()

    # Divide o texto em palavras
    words = text.split()

    # Remove as stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Realiza a stemming das palavras
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # Retorna as palavras pré-processadas como uma lista
    return words


# Função para calcular o IDF (Inverse Document Frequency) de um termo
def calculate_idf(term, doc_dict):

    aux = sum(1 for doc_text in doc_dict.values() if term in doc_text)
    num_docs_with_term = aux

    if num_docs_with_term == 0:
        return 0
    else:
        return math.log(len(doc_dict)/num_docs_with_term)


# Função para calcular o BM25 score de um documento para uma consulta
def calculate_bm25_score(doc_id, query_terms, doc_dict, k1=1.2, b=0.75):

    doc_text = doc_dict[doc_id]
    doc_len = len(doc_text)

    aux = sum(len(doc_text) for doc_text in doc_dict.values()) / len(doc_dict)
    avg_doc_len = aux

    doc_terms = preprocess_text(doc_text)

    query_term_counts = Counter(query_terms)
    score = 0
    for term in set(query_terms):
        if term not in doc_terms:
            continue
        tf = doc_terms.count(term)
        idf = calculate_idf(term, doc_dict)
        query_tf = query_term_counts[term]
        score += idf * ((k1 + 1)*tf) / (k1*((1 - b) + b*(doc_len/avg_doc_len))
                        + tf)*query_tf

    return score


# Função para classificar todos os documentos da coleção em ordem decrescente
#   de BM25 score para uma consulta
def rank_documents(query, doc_dict):

    query_terms = preprocess_text(query)

    doc_scores = {}
    for doc_id, doc_text in doc_dict.items():
        doc_scores[doc_id] = calculate_bm25_score(doc_id, query_terms,
                                                  doc_dict)
        sorted_doc_scores = sorted(doc_scores.items(), key=lambda x: x[1],
                                   reverse=True)
        ranked_docs = [(doc_id, score) for doc_id, score in sorted_doc_scores]

    return ranked_docs


# Função para executar buscas a partir de dicionários de documentos e queries,
#   especificando a quantidade de documentos a serem retornados por query
def executa_buscas(docs_dict, qrys_dict, qtd_returns, imprime_log):

    qrys_return = {}
    for qry_id, qry_text in qrys_dict.items():
        
        if imprime_log:
            print(f'Processando Query {qry_id}\n')
        
        doc_scores = rank_documents(qry_text, docs_dict)

        qrys_return[qry_id] = []
        for doc_id, score in doc_scores[:10]:
            qrys_return[qry_id].append(doc_id)
            if imprime_log:
                print(f'Doc ID: {doc_id}')
                print(f'Score: {score:.4f}')
                print(f'Text: {docs_dict[doc_id]}')
                print('-'*50)
        
        if imprime_log:
            print('')
    
    return qrys_return

## Execução das buscas

In [11]:
# Transforma documentos e queries no formato a ser inserido no código de busca

docs_input = {}
for doc_id, doc_dict in docs_dict.items():
    doc_input = (doc_dict.get('Titulo', '') + ' '
                 + doc_dict.get('Autor', '') + ' '
                 + doc_dict.get('Referencia', '') + ' '
                 + doc_dict.get('Texto', ''))
    docs_input[doc_id] = doc_input.strip()

qrys_input = {}
for qry_id, qry_dict in qrys_dict.items():
    qry_input = (qry_dict.get('Titulo', '') + ' '
                 + qry_dict.get('Autor', '') + ' '
                 + qry_dict.get('Referencia', '') + ' '
                 + qry_dict.get('Texto', ''))
    qrys_input[qry_id] = qry_input.strip()

In [12]:
# Processa as buscas, retornando os 10 documentos mais relevantes

qrys_return = executa_buscas(docs_dict=docs_input,
                             qrys_dict=qrys_input,
                             qtd_returns=10,
                             imprime_log=False)

## Verificação do resultado de uma busca

In [13]:
# Mostra os outputs intermediários da busca para a query 1, como teste

qrys_input_teste = {}
qrys_input_teste[1] = qrys_input[1]

qrys_return_teste = executa_buscas(docs_dict=docs_input,
                                   qrys_dict=qrys_input_teste,
                                   qtd_returns=10,
                                   imprime_log=True)

qrys_return_teste

Processando Query 1

Doc ID: 429
Score: 25.3735
Text: The Information Content of Titles in Engineering Literature Bottle, Robert T.  Since many alerting and information services rely very heavily on the use of titles to transfer information to the potential user, it is essential that  he be aware of the proportion of the information contained in the complete document which will not be deducible from the title and which he will therefore miss.. Methods will be discussed for analyzing the relative information content of the titles of engineering paper and results presented for the amount and  type of information lost through scanning title listing only..    Between one-third and one-half of indexable terms are not retrievable from article titles even if all possible synonyms and  related terms are used.. If all synonyms are used instead of one keyword the amount of information  retrieved is increased by about 70 percent.. The problems of dealing with synonyms and with syntactical variant

{1: [429, 722, 1299, 759, 65, 928, 820, 603, 1055, 76]}

## Comparação dos resultados com a prova real

In [14]:
# Calcula a métrica de precisão para cada query

precisao_por_qry = {}
for qry_id, return_list in qrys_return.items():

    tp = 0          # Verdadeiros positivos
    fp = 0          # Falsos positivos

    if qry_id not in rels_dict:
        precisao_por_qry[qry_id] = 0
        continue

    for doc_id in return_list:
        if str(doc_id) in rels_dict[qry_id]['Doc IDs']:
            tp += 1
        else:
            fp += 1

    precisao_por_qry[qry_id] = tp/(tp + fp)

In [15]:
# Calcula a precisão média entre todas as queries

precisao_media = sum(precisao_por_qry.values())/len(precisao_por_qry)

precisao_media

0.23660714285714293